# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Time_Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit e209e243b41e8e97388d05d394ba40248e6f2115
Author: Najwa Ez Zine <nezzine@fnancy>
Date:   Mon May 28 15:55:00 2018 +0200

    Working on S[]


Python 2.7.13


#### Préparation de l'environnement

In [10]:
%%bash -s
pip3 install --user execo
pip3 install --user requests

  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a94236f6d4b8223e21beb4d576dc72e8130bd7880f600839b8/urllib3-1.22-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl


In [11]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output

##### Réservation noeuds

In [16]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 5
# Temps réservation
walltime = "5:0:0" 

# filters out Nancy's graphene cluster (buggy)
resources_selection = "-p \"cluster != 'econome'\"" 

# Nancy
site = "nantes"

In [17]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 91346


On les range

In [19]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('ecotype-22.nantes.grid5000.fr'),
 Host('ecotype-25.nantes.grid5000.fr'),
 Host('ecotype-27.nantes.grid5000.fr'),
 Host('ecotype-3.nantes.grid5000.fr'),
 Host('ecotype-32.nantes.grid5000.fr')]

##### Déploiement de l'environnement

In [28]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [29]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

nantes: Deployment #D-fc8ab057-2428-4c5e-9ce3-914af330d095 started
nantes: Grab the key file /home/nezzine/.ssh/authorized_keys
nantes: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
nantes: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
nantes: Launching a deployment on ecotype-[3,22,25,27,32].nantes.grid5000.fr
nantes: Performing a Deploy[SetDeploymentEnvUntrusted] step
nantes:   switch_pxe
nantes:   reboot
nantes:    * Performing a soft reboot on ecotype-[3,22,25,27,32].nantes.grid5000.fr
nantes:   wait_reboot
nantes:   send_key_in_deploy_env
nantes:   create_partition_table
nantes:   format_deploy_part
nantes:   mount_deploy_part
nantes:   format_tmp_part
nantes:   format_swap_part
nantes: End of step Deploy[SetDeploymentEnvUntrusted] after 184s
nantes: Performing a Deploy[BroadcastEnvKascade] step
nantes:   send_environment
nantes:    * Broadcast time: 75s
nantes:   manage_admin_post_install
nantes:

nantes: Warning: Permanently added 'nantes.grid5000.fr,172.16.207.100' (ECDSA) to the list of known hosts.
nantes: Connection to nantes.grid5000.fr closed.


#### Test : commande simple 'ls'

In [ ]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

#### Installation environnement sur noeud

In [30]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [31]:
Remote_install.run().ok

True

#### Version Simgrid

In [32]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [33]:
path = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name+'/src/'

#### Récupération fichiers sources

In [34]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

True

In [35]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path+"griffon.xml",path+"griffon_hostfile.txt",path+"broadcast.c",path+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

True

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'+experiment_name+'/results/'+datetime.datetime.now().strftime("%Y-%m-%d"))
Result_get.run().ok

In [ ]:
for i in range(0,nodecount) :
    execo.action.Get(hosts=nodes[i],remote_files=["~/experiments_src/bcast_results.txt"], local_location='~/Stage-POLARIS/SimGrid/examples/SMPI/'+experiment_name+'/results/'+datetime.datetime.now().strftime("%Y-%m-%d")+'/'+get_host_shortname(nodes[i])+'_'+datetime.datetime.now().strftime("%Y-%m-%d_%H:%M")+'.txt',connection_params={'user':'root'}).run().ok

##### Shortcut

In [43]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes, local_files=[path
                                   + 'griffon.xml', path
                                   + 'griffon_hostfile.txt', path
                                   + 'broadcast.c', path + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})

def parl_exec () :
    # Nbr pcs
    NPROCS = 2
    # Nbr itérations
    N = 10
    D = [1,3]
    for i in range (0,nodecount) :
       print("i = "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str((int)((i*N)/len(nodes)))
             +" MAX = "+str((int)(((i+1)*N)/len(nodes)))
             +" N="+str(N)+" DELAY='"+str(D[0])+" "+str(D[1])+"'")
       bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str((int)((i*N)/len(nodes))) 
                                  +' MAX='+str((((i+1)*N)/len(nodes)))+' N='+str(N)
                                  +" DELAY='"+str(D[0])+" "+str(D[1])
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
       print(bool)
    return;

# Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10',hosts=nodes,connection_params={'user':'root'})

Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))

Src_get.run().ok
Remote_smpi_env.run().ok

# Remote_smpi.run().ok

parl_exec()
Result_get.run().ok
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location='~/Stage-POLARIS/SimGrid/examples/SMPI/'
                      + experiment_name + '/results/'
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

i = 0 NPROCS = 2 MIN = 0 MAX = 2 N=10 DELAY='1 3'
True
i = 1 NPROCS = 2 MIN = 2 MAX = 4 N=10 DELAY='1 3'
True
i = 2 NPROCS = 2 MIN = 4 MAX = 6 N=10 DELAY='1 3'
True
i = 3 NPROCS = 2 MIN = 6 MAX = 8 N=10 DELAY='1 3'
True
i = 4 NPROCS = 2 MIN = 8 MAX = 10 N=10 DELAY='1 3'
True


#### Publication des résultats

In [40]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'-'"$2"

[master 7ce6aa8] Automatic results update : Time_Bcast-nantes
 Committer: Najwa Ez Zine <nezzine@fnantes-stretch>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 7 files changed, 661 insertions(+), 905 deletions(-)
 rewrite SimGrid/examples/SMPI/Time_Bcast/Time Broadcast.ipynb (75%)
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-05-29/ecotype-22_2018-05-29_12:54.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-05-29/ecotype-25_2018-05-29_12:54.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-05-29/ecotype-27_2018-05-29_12:54.txt
 create mode 100

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)